In [1]:
import numpy as np
from collections import deque

In [2]:
def get_citation_graph(file_path):
    adjacency_list = {}
    node_list = set()

    with open(file_path, 'r') as file:
        for line in file:
            # Split the line into source and target vertices
            target, source = map(int, line.strip().split())
            node_list.add(source)
            node_list.add(target)

            # Add the edge to the adjacency list
            if source in adjacency_list:
                adjacency_list[source].append(target)
            else:
                adjacency_list[source] = [target]

    return adjacency_list, node_list

def add_sink_nodes(graph, node_list):
    graph_nodes = list(graph.keys())
    sink_nodes = []
    for node in node_list:
        if node not in graph_nodes:
            # graph[node] = []
            sink_nodes.append(node)

    for node in sink_nodes:
        add_nodes = []
        for key, values in graph.items():
            if node in values:
                add_nodes.append(key)
        graph[node] = add_nodes

    return graph

def find_shortest_paths(start, end, graph, dist):
    if start == end:
        return [[start]]
    
    paths = []
    for neighbor in graph[start]:
        if dist[start][end] == dist[neighbor][end] + 1:
            for path in find_shortest_paths(neighbor, end, graph, dist):
                paths.append([start] + path)
    
    return paths

# def all_shortest_paths(graph):
#     INF = float('inf')
#     n = len(graph)
#     nodeList = list(graph.keys())
    
#     # Initialize the distance matrix
#     # dist = [[INF] * n for _ in range(n)]
#     dist = {}

#     for node in nodeList:
#         temp = {}
#         for adj in nodeList:
#             temp[adj] = INF
#         dist[node] = temp

#     for i in nodeList:
#         dist[i][i] = 0
#         for neighbor in graph[i]:
#             dist[i][neighbor] = 1  # Assuming every edge cost is 1
    
#     print('Running Floyd-Warshall ...\n')
#     # Floyd-Warshall algorithm
#     for k in nodeList:
#         for i in nodeList:
#             for j in nodeList:
#                 dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    
#     print('Extracting shortest paths ... \n')
#     # Extract all shortest paths
#     #paths = [[[] for _ in nodeList] for _ in nodeList]
#     paths = {}

#     for node in nodeList:
#         temp = {}
#         for adj in nodeList:
#             temp[adj] = []
#         paths[node] = temp

#     for i in nodeList:
#         for j in nodeList:
#             if dist[i][j] != INF:
#                 paths[i][j] = find_shortest_paths(i, j, graph, dist)
    
#     return paths, dist

def shortest_paths_bfs(graph, start):
    # Initialize distances dictionary to store the shortest distances
    distances = {node: float('inf') for node in graph}
    
    # Set the distance to the start node as 0
    distances[start] = 0
    
    # Initialize a queue for BFS
    queue = deque([start])

    while queue:
        current_node = queue.popleft()

        for neighbor in graph[current_node]:
            if distances[neighbor] == float('inf'):
                # If the distance is infinite, update it and enqueue the neighbor
                distances[neighbor] = distances[current_node] + 1
                queue.append(neighbor)

    return distances

def all_shortest_paths(graph):
    INF = float('inf')
    dist = {}
    nodeList = list(graph.keys())
    
    # Iterate through all nodes and find shortest paths from each node
    print('Retrieving shortest path distances ...\n')
    for node in graph:
        dist[node] = shortest_paths_bfs(graph, node)

    print('Extracting shortest paths ... \n')
    # Extract all shortest paths
    #paths = [[[] for _ in nodeList] for _ in nodeList]
    paths = {}

    for node in nodeList:
        temp = {}
        for adj in nodeList:
            temp[adj] = []
        paths[node] = temp

    for i in nodeList:
        for j in nodeList:
            if dist[i][j] != INF:
                paths[i][j] = find_shortest_paths(i, j, graph, dist)
    
    return paths, dist

def get_closeness_centrality(distance_dict):
    n = len(distance_dict)
    centrality_scores = {}

    for node, distances in distance_dict.items():
        for key, value in distances.items():
            if value == float('inf'):
                distance_dict[node][key] = 0
                
        total_distance = sum(list(distance_dict[node].values()))
        closeness_centrality = (n - 1) / total_distance if total_distance != 0 else 0
        centrality_scores[node] = closeness_centrality

    centrality_scores = {k: v for k, v in sorted(centrality_scores.items(), key=lambda item: item[1], reverse = True)}

    return centrality_scores

def get_betweenness_centrality(shortest_path_list):

    nodeList = list(shortest_path_list.keys())
    n = len(nodeList)
    # print('Computing shortest paths between each pair of node ...\n')
    # shortest_path_list = all_shortest_paths(graph)
    bet_centrality = {}

    for node in nodeList:
        # print(f'Computing betweenness centrality for node {node}\n')
        centrality = 0
        for i, pathList in shortest_path_list.items(): # list of all shortest paths for each node i to all node j
            for j, shortest_paths in pathList.items(): # list of all shortest paths from node i to node j
                tot_path_i_j = len(shortest_paths)
                i_node_j = 0 
                for path in shortest_paths: # one of the shortest path from i to j
                    path_len = len(path)
                    if node in path and node != path[0] and node != path[path_len-1]:
                        i_node_j += 1

                centrality += i_node_j / tot_path_i_j if tot_path_i_j != 0 else 0

        bet_centrality[node] = centrality / ((n-1) * (n-2))

    bet_centrality = {k: v for k, v in sorted(bet_centrality.items(), key=lambda item: item[1], reverse = True)}
    return bet_centrality

def get_pagerank(graph, damping_factor=0.8, epsilon=1e-8, max_iterations=100):
    nodeList = list(graph.keys())
    num_nodes = len(nodeList)
    node_dir = {}

    i = 0
    for node in nodeList:
        node_dir[node] = i
        i += 1

    transition_matrix = np.zeros((num_nodes, num_nodes))

    for node, neighbors in graph.items():
        if neighbors:
            num_neighbors = len(neighbors)
            for neighbor in neighbors:
                x = node_dir[neighbor]
                y = node_dir[node]
                transition_matrix[x, y] = 1 / num_neighbors

    # Damping factor
    damping_matrix = (1 - damping_factor) / num_nodes * np.ones((num_nodes, num_nodes))
    transition_matrix = damping_factor * transition_matrix + damping_matrix

    # Initial PageRank values
    page_rank_vector = np.ones(num_nodes) / num_nodes

    # Power iteration method to calculate PageRank
    for i in range(max_iterations):
        # print('Iteration:', i)
        new_page_rank = np.dot(transition_matrix, page_rank_vector)
        if np.linalg.norm(new_page_rank - page_rank_vector, ord=1) < epsilon:
            break
        page_rank_vector = new_page_rank

    page_rank_final = {}
    for node in nodeList:
        page_rank_final[node] = page_rank_vector[node_dir[node]]

    page_rank_final = {k: v for k, v in sorted(page_rank_final.items(), key=lambda item: item[1], reverse = True)}
    
    return page_rank_final

def write_file(file_name, data_dict):
    file_path = './' + file_name
    with open(file_path, 'w') as file:
        for node, centrality in data_dict.items():
            file.write('{} {:.6f}\n'.format(node, centrality))

In [7]:
file_path = './demo.txt'
# file_path = './cora/cora.cites'
print('Fetching the graph ...\n')
citation_network, node_list = get_citation_graph(file_path)
citation_network = add_sink_nodes(citation_network, node_list)

Fetching the graph ...



In [8]:
# shortest_paths, dist = all_shortest_paths(citation_network)
shortest_paths, dist = all_shortest_paths(citation_network)

Retrieving shortest path distances ...

Extracting shortest paths ... 



In [9]:
shortest_paths

{1: {1: [[1]], 2: [[1, 0, 2]], 3: [], 0: [[1, 0]]},
 2: {1: [[2, 1]], 2: [[2]], 3: [], 0: [[2, 0]]},
 3: {1: [[3, 1]], 2: [[3, 2]], 3: [[3]], 0: [[3, 2, 0], [3, 1, 0]]},
 0: {1: [[0, 1]], 2: [[0, 2]], 3: [], 0: [[0]]}}

In [5]:
closeness_centrality = get_closeness_centrality(dist)
write_file('closeness.txt', closeness_centrality)

In [6]:
betweenness_centrality = get_betweenness_centrality(shortest_paths)
write_file('beetweenness.txt', betweenness_centrality)

In [20]:
pagerank = get_pagerank(citation_network)
write_file('pagerank.txt', pagerank)

In [20]:
import pickle
# with open('distance.pickle', 'wb') as handle:
#     pickle.dump(dist, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('shortest_paths.pickle', 'wb') as handle:
#     pickle.dump(shortest_paths, handle, protocol=pickle.HIGHEST_PROTOCOL)



with open('distance.pickle', 'rb') as handle:
    dist = pickle.load(handle)

with open('shortest_paths.pickle', 'rb') as handle:
    shortest_paths = pickle.load(handle)